In [115]:
from chronos import BaseChronosPipeline, Chronos2Pipeline
import pandas as pd
import os
from dotenv import load_dotenv
import requests
from utils import upload
import dropbox
from pandas.tseries.frequencies import to_offset
import holidays
import torch

load_dotenv()

True

In [116]:
try:
    chronos_gpu_or_cpu = os.getenv("CHRONOS_GPU_OR_CPU", "gpu").lower()
    if chronos_gpu_or_cpu not in ["gpu", "cpu"]:
        raise ValueError("Invalid value for CHRONOS_GPU_OR_CPU. Must be 'gpu' or 'cpu'.")

    if chronos_gpu_or_cpu == "gpu":


        torch.cuda.set_device(int(os.getenv("CHRONOS_GPU_DEVICE", 0)))

        # Load the Chronos-2 pipeline
        # GPU recommended for faster inference, but CPU is also supported
        pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
            "amazon/chronos-2",
            device_map="cuda"
            # device_map="cpu"
    )
    else:
        
        # Load the Chronos-2 pipeline
        # GPU recommended for faster inference, but CPU is also supported
        pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
            "amazon/chronos-2",
            device_map="cpu"
    )
except Exception as e:
    print(f"Error loading Chronos-2 pipeline: {e}")
    
    pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-2",
    device_map="cpu")
    raise


In [117]:
def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

def add_holiday_flags(
    df: pd.DataFrame,
    ts_col: str = "ds",
    local_tz: str = "America/Montreal",
    observed: bool = True,
    include_names: bool = False,
) -> pd.DataFrame:
    """
    Adds boolean columns:
      • is_qc_holiday       — Québec public holiday (CA-QC)
      • is_jewish_holiday   — Israeli public/Jewish holiday (IL)
    Optionally adds:
      • qc_holiday_name
      • jewish_holiday_name

    Notes:
      • Holiday checks are date-based (00:00–24:00 local calendar date),
        not sundown-to-sundown observance.
      • NaT timestamps are ignored gracefully.
    """
    out = df.copy()

    # 1) Parse to datetime
    out[ts_col] = pd.to_datetime(out[ts_col], errors="coerce")

    # 2) Get the calendar DATE to use for holiday lookup
    #    - If tz-aware: convert to Montreal then take .date
    #    - If naive: assume values already represent local Montreal wall-clock; just take .date
    if getattr(out[ts_col].dt, "tz", None) is not None:
        dates_for_calendar = out[ts_col].dt.tz_convert(local_tz).dt.date
    else:
        dates_for_calendar = out[ts_col].dt.date

    # 3) Build a SAFE integer year range for the holiday objects
    years_series = pd.Series(dates_for_calendar)
    years_series = years_series.dropna().map(lambda d: int(pd.Timestamp(d).year))
    if years_series.empty:
        raise ValueError("No valid datetimes found to extract holiday years.")
    years = list(range(int(years_series.min()), int(years_series.max()) + 1))

    # 4) Construct holiday calendars
    qc_holidays = holidays.Canada(subdiv="QC", years=years, observed=observed)
    il_holidays = holidays.Israel(years=years, observed=observed)

   # 5) Flag membership
    out["is_qc_holiday"] = [ ("yes" if d in qc_holidays else "no") if pd.notna(pd.Timestamp(d)) else "no"
                             for d in dates_for_calendar ]
    out["is_jewish_holiday"] = [ ("yes" if d in il_holidays else "no") if pd.notna(pd.Timestamp(d)) else "no"
                                 for d in dates_for_calendar ]

    if include_names:
        out["qc_holiday_name"] = [ qc_holidays.get(d, "no") if pd.notna(pd.Timestamp(d)) else "no"
                                   for d in dates_for_calendar ]
        out["jewish_holiday_name"] = [ il_holidays.get(d, "no") if pd.notna(pd.Timestamp(d)) else "no"
                                       for d in dates_for_calendar ]

    return out

shift_types_dict = {'W1':'flow',
 'X1':'pod',
 'X3':'pod',
 'X4':'vertical',
 'X2':'vertical',
 'WOC1':'oncall',
 'WOC2':'oncall',
 'WOC3':'oncall',
 'X5':'pod',
 'W3':'overlap',
 'Y1':'pod',
 'Y3':'pod',
 'Y4':'vertical',
 'Y2':'vertical',
 'Y5':'pod',
 'Z1':'night',
 'Z2':'night',
 'D1':'pod',
 'R1':'pod',
 'P1':'vertical',
 'D2':'vertical',
 'OC1':'oncall',
 'OC2':'oncall',
 'V1':'flow',
 'A1':'pod',
 'G1':'vertical',
 'E1':'pod',
 'R2':'pod',
 'A2':'pod',
 'P2':'vertical',
 'E2':'vertical',
 'N1':'night',
 'N2':'night',
 'L2':'overlap',
 'L4':'overlap',
 'H1':'teaching',
 'B1':'vertical',
 'L1':'overlap',
 'W5':'overlap',
 'L6':'overlap',
 'B2':'vertical'}

In [118]:
# Load hourly data
df = pd.read_csv(
    'https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id'] = 'jgh'

hourly_report_df = df.copy()
df.tail()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM,total_tbs,vert_tbs,pod_tbs,overflow,id
44874,2026-02-18 17:00:00,5,103,3,73,8,176,0,24,0,...,0,4,3,12,8,31,27,4,12,jgh
44875,2026-02-18 18:00:00,9,112,3,76,12,188,1,25,0,...,0,1,1,11,8,35,32,3,9,jgh
44876,2026-02-18 19:00:00,14,126,2,78,16,204,5,30,0,...,0,0,3,9,8,32,24,8,13,jgh
44877,2026-02-18 20:00:00,7,133,3,81,10,214,3,33,0,...,0,0,1,9,7,34,21,13,13,jgh
44878,2026-02-18 21:00:00,7,140,3,84,10,224,4,37,0,...,0,0,4,9,7,27,16,11,14,jgh


In [119]:
# Load shift data
all_shifts_df = pd.read_csv('https://www.dropbox.com/scl/fi/yeyr2a7pj6nry8i2q3m0c/all_shifts.csv?rlkey=q1su2h8fqxfnlu7t1l2qe1w0q&raw=1')
all_shifts_df['shift_start'] = pd.to_datetime(all_shifts_df['shift_start']).dt.round('h')
all_shifts_df['shift_end'] = pd.to_datetime(all_shifts_df['shift_end']).dt.round('h')
all_shifts_df['shift_type'] = all_shifts_df['shift_short_name'].map(shift_types_dict)
all_shifts_df.tail()

,scheduled_shift_id,group_id,user_id,employee_id,npi,first_name,last_name,facility_id,facility_ext_id,facility_name,...,shift_start,shift_end,shift_hours,work_start,work_end,work_hours,count_as_shift,is_night,is_weekend,shift_type
29996,88886,1,28,NaN,NaN,Emily,Moras,1,NaN,Jewish General Hospital,...,2026-02-26 16:00:00,2026-02-27 00:00:00,8.00,2/26/2026 16:00,2/27/2026 00:00,8.00,1,0,0,vertical
29997,88884,1,73,NaN,NaN,Madison,Meehan,1,NaN,Jewish General Hospital,...,2026-02-26 16:00:00,2026-02-27 00:00:00,8.00,2/26/2026 16:00,2/27/2026 00:00,8.00,1,0,0,pod
29998,88887,1,80,NaN,NaN,Natasha,Poirier,1,NaN,Jewish General Hospital,...,2026-02-26 18:00:00,2026-02-27 02:00:00,8.00,2/26/2026 18:00,2/27/2026 02:00,8.00,1,0,0,vertical
29999,88219,1,12,NaN,NaN,Julia,Bernard,1,NaN,Jewish General Hospital,...,2026-02-27 00:00:00,2026-02-27 08:00:00,8.08,2/26/2026 23:45,2/27/2026 07:50,8.08,1,1,0,night
30000,88888,1,61,NaN,NaN,Alexander,Hart,1,NaN,Jewish General Hospital,...,2026-02-27 00:00:00,2026-02-27 08:00:00,8.08,2/26/2026 23:45,2/27/2026 07:50,8.08,1,1,0,night


In [120]:
# Create hourly rows
# We'll use a list comprehension to generate the range for each row
expanded_rows = []
for _, row in all_shifts_df.iterrows():
    # Create range. inclusive='left' means [start, end)
    # If start == end (e.g. 0 length shift after rounding), it will be empty, which is correct
    hours = pd.date_range(row['shift_start'], row['shift_end'], freq='h', inclusive='left')
    for h in hours:
        expanded_rows.append({
            'ds': h,
            'user': row['first_name']+row['last_name'],
            'shift_type': row['shift_type'],
            'shift_short_name': row['shift_short_name']
        })

expanded_df = pd.DataFrame(expanded_rows)

expanded_df.tail()

,ds,user,shift_type,shift_short_name
273246,2026-02-27 03:00:00,AlexanderHart,night,N2
273247,2026-02-27 04:00:00,AlexanderHart,night,N2
273248,2026-02-27 05:00:00,AlexanderHart,night,N2
273249,2026-02-27 06:00:00,AlexanderHart,night,N2
273250,2026-02-27 07:00:00,AlexanderHart,night,N2


In [121]:
# Pivot
# index=timestamp, columns=user_id, values=shift_type
hourly_shifts_by_user_df = expanded_df.pivot_table(
    index='ds', 
    columns='user', 
    values='shift_type', 
    aggfunc='first' # In case of duplicates, take the first
)

# Fill NaNs
hourly_shifts_by_user_df = hourly_shifts_by_user_df.fillna('NotWorking')
hourly_shifts_by_user_df.tail()

user,AlanAzuelos,AlexGuttman,AlexanderHart,AlexandreDostaler,AlexisHaligua,AmelieBellemare,ArzuChaudhry,BernardUnger,DahliaGuttman,DanielMankarios,...,PaulBrisebois,PhilipStasiak,RafaelAroutiunian,SaraAhronheim,ShuoPeng,StephenRosenthal,TanLe,ThuHangTran,WayneChoi,WillGrad
ds,,,,,,,,,,,,,,,,,,,,,
2026-02-27 03:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-27 04:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-27 05:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-27 06:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2026-02-27 07:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking


In [122]:
df.columns.tolist()

['ds',
 'INFLOW_STRETCHER',
 'Infl_Stretcher_cum',
 'INFLOW_AMBULATORY',
 'Infl_Ambulatory_cum',
 'Inflow_Total',
 'Inflow_Cum_Total',
 'INFLOW_AMBULANCES',
 'Infl_Ambulances_cum',
 'FLS',
 'CUM_ADMREQ',
 'CUM_BA1',
 'WAITINGADM',
 'TTStr',
 'TRG_HALLWAY1',
 'TRG_HALLWAY_TBS',
 'reoriented_cum',
 'reoriented_cum_MD',
 'QTRACK1',
 'RESUS',
 'Pod_T',
 'POD_GREEN',
 'POD_GREEN_TBS',
 'POD_YELLOW',
 'POD_YELLOW_TBS',
 'POD_ORANGE',
 'POD_ORANGE_TBS',
 'POD_CONS_MORE2H',
 'POD_IMCONS_MORE4H',
 'POD_XRAY_MORE2H',
 'POD_CT_MORE2H',
 'POST_POD1',
 'VERTSTRET',
 'RAZ_TBS',
 'RAZ_LAZYBOY',
 'RAZ_WAITINGREZ',
 'AMBVERT1',
 'AMBVERTTBS',
 'QTrack_TBS',
 'Garage_TBS',
 'RAZ_CONS_MORE2H',
 'RAZ_IMCONS_MORE4H',
 'RAZ_XRAY_MORE2H',
 'RAZ_CT_MORE2H1',
 'PSYCH1',
 'PSYCH_WAITINGADM',
 'total_tbs',
 'vert_tbs',
 'pod_tbs',
 'overflow',
 'id']

In [123]:
ID_COL = "id"
TS_COL = "ds"
# TARGETS = ['total_tbs', 'Inflow_Total', 'overflow']
# Targets are all columns in df except ds (timestamp) and id
TARGETS = [col for col in df.columns.tolist() if col != TS_COL and col != ID_COL]

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates(
    [ID_COL, TS_COL], keep="last")





# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")

In [124]:
# Predict
print('Predicting basic forecast')
basic_forecast = pipeline.predict_df(
    df,
    prediction_length=24,
    # future_df = future_df.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

basic_forecast.head()

Predicting basic forecast


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,jgh,2026-02-18 22:00:00,INFLOW_STRETCHER,5.088941,2.657592,3.411573,3.994172,4.539448,5.088941,5.684533,6.303296,7.078827,8.186040
1,jgh,2026-02-18 23:00:00,INFLOW_STRETCHER,4.261419,1.943836,2.629474,3.181258,3.712052,4.261419,4.854199,5.481572,6.270520,7.399102
2,jgh,2026-02-19 00:00:00,INFLOW_STRETCHER,3.503133,1.518808,2.097980,2.571852,3.026967,3.503133,4.010885,4.583101,5.351997,6.461112
3,jgh,2026-02-19 01:00:00,INFLOW_STRETCHER,2.976579,1.128704,1.635174,2.077600,2.525173,2.976579,3.448426,3.959908,4.656681,5.653827
4,jgh,2026-02-19 02:00:00,INFLOW_STRETCHER,2.313057,0.857733,1.265550,1.621304,1.958593,2.313057,2.729743,3.188126,3.812223,4.753717


In [125]:
basic_forecast.target_name.value_counts()

target_name
INFLOW_STRETCHER       24
Infl_Stretcher_cum     24
INFLOW_AMBULATORY      24
Infl_Ambulatory_cum    24
Inflow_Total           24
Inflow_Cum_Total       24
INFLOW_AMBULANCES      24
Infl_Ambulances_cum    24
FLS                    24
CUM_ADMREQ             24
CUM_BA1                24
WAITINGADM             24
TTStr                  24
TRG_HALLWAY1           24
TRG_HALLWAY_TBS        24
reoriented_cum         24
reoriented_cum_MD      24
QTRACK1                24
RESUS                  24
Pod_T                  24
POD_GREEN              24
POD_GREEN_TBS          24
POD_YELLOW             24
POD_YELLOW_TBS         24
POD_ORANGE             24
POD_ORANGE_TBS         24
POD_CONS_MORE2H        24
POD_IMCONS_MORE4H      24
POD_XRAY_MORE2H        24
POD_CT_MORE2H          24
POST_POD1              24
VERTSTRET              24
RAZ_TBS                24
RAZ_LAZYBOY            24
RAZ_WAITINGREZ         24
AMBVERT1               24
AMBVERTTBS             24
QTrack_TBS             24


In [126]:
# basic_forecast


df_with_holidays = add_holiday_flags(df, ts_col='ds', include_names=True)

#create a dataframe with the next 24 hours timestamps hourly as column 'ds', with column 'id' jgh
future_df = hourly_shifts_by_user_df.reset_index()[hourly_shifts_by_user_df.reset_index()['ds'] > df['ds'].max()]
future_df['id'] = 'jgh'
future_df = add_holiday_flags(future_df, ts_col='ds', include_names=True)

# First, add holiday flags to future_df
future_df_with_added_holidays = add_holiday_flags(future_df, ts_col='ds', include_names=True)

# Then, select only the columns from future_df_with_added_holidays that are also in df_with_holidays
common_columns = [col for col in future_df_with_added_holidays.columns if col in df_with_holidays.columns]
future_df_with_holidays = future_df_with_added_holidays[common_columns]

# Predict
print('Predicting forecast with holidays')  
forecast_with_holidays = pipeline.predict_df(
    df_with_holidays,
    prediction_length=24,
    future_df = future_df_with_holidays.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)
forecast_with_holidays.tail()

Predicting forecast with holidays


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
1171,jgh,2026-02-19 17:00:00,overflow,13.763207,6.389228,8.601532,10.416212,12.086449,13.763207,15.625778,17.591282,20.008766,23.202866
1172,jgh,2026-02-19 18:00:00,overflow,12.476545,5.873883,7.735642,9.327193,10.881773,12.476545,14.196137,16.051090,18.430012,21.651127
1173,jgh,2026-02-19 19:00:00,overflow,11.385977,5.377185,7.217317,8.674048,10.015683,11.385977,12.893236,14.529823,16.672981,19.788464
1174,jgh,2026-02-19 20:00:00,overflow,10.292620,4.860961,6.536765,7.844835,9.046489,10.292620,11.674438,13.202819,15.164167,17.949821
1175,jgh,2026-02-19 21:00:00,overflow,9.420338,4.530518,6.017988,7.176142,8.271197,9.420338,10.719385,12.145000,13.997894,16.709957


In [127]:
df_with_staffing = df.merge(hourly_shifts_by_user_df, on='ds')
future_df_with_staffing = hourly_shifts_by_user_df.reset_index()[hourly_shifts_by_user_df.reset_index()['ds'] > df['ds'].max()]
future_df_with_staffing['id'] = 'jgh'

print('Predicting forecast with staffing')
forecast_with_staffing = pipeline.predict_df(
    df_with_staffing,
    prediction_length=24,
    future_df = future_df_with_staffing.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    # quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

forecast_with_staffing.tail()

Predicting forecast with staffing


,id,ds,target_name,predictions,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
1171,jgh,2026-02-19 17:00:00,overflow,14.323666,5.946801,8.419822,10.526567,12.497286,14.323666,16.292961,18.305632,20.812330,24.091545
1172,jgh,2026-02-19 18:00:00,overflow,13.235572,5.507154,7.687353,9.627621,11.490952,13.235572,15.082417,17.105003,19.707783,23.036583
1173,jgh,2026-02-19 19:00:00,overflow,12.330551,5.041592,7.308933,9.161131,10.804455,12.330551,14.039944,15.870131,18.246872,21.585052
1174,jgh,2026-02-19 20:00:00,overflow,11.497987,4.799321,6.837901,8.488470,10.020679,11.497987,13.108173,14.857849,17.035519,19.958622
1175,jgh,2026-02-19 21:00:00,overflow,10.453028,4.546135,6.388125,7.802498,9.110331,10.453028,11.930269,13.538686,15.591499,18.460098


In [128]:
weather_df = pd.read_csv('https://www.dropbox.com/scl/fi/gmhwwld9z9yychg4r0yuk/weather.csv?rlkey=66c78m90aviamr0x0uu72pfr8&raw=1')
weather_df.ds = pd.to_datetime(weather_df.ds, errors="coerce")


future_weather_df = weather_df[weather_df.ds > df.ds.max()].head(24)
future_weather_df['id']='jgh'

print('Predicting forecast with weather')
# Predict
forecast_with_weather = pipeline.predict_df(
    #join df with weather_df on ds
    df.merge(weather_df, on='ds'),
    prediction_length=24,
    #weather_df where ds is greater than the max of df.ds.max()
    future_df = future_weather_df,
    # future_df = future_df.head(24),
    # quantile_levels=[0.1, 0.5, 0.9],
    quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)

Predicting forecast with weather


In [129]:
weather_df.tail()

,ds,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,wind_speed_10m,wind_direction_10m,wind_gusts_10m,precipitation_probability
45144,2026-02-25 00:00:00,-9.611000,84.0,-11.807982,-14.734683,0.0,0.0,0.0,0.24,3.0,1015.299988,986.067017,100.0,100.0,100.0,10.0,11.901798,273.468170,32.399998,3.0
45145,2026-02-25 01:00:00,-10.311000,83.0,-12.643393,-15.344110,0.0,0.0,0.0,0.24,3.0,1015.900024,986.573364,100.0,100.0,100.0,3.0,10.895576,277.594543,30.239998,2.0
45146,2026-02-25 02:00:00,-10.960999,82.0,-13.429729,-15.948289,0.0,0.0,0.0,0.24,3.0,1016.500000,987.084473,100.0,99.0,100.0,0.0,10.239453,280.124603,27.000000,2.0
45147,2026-02-25 03:00:00,-11.710999,82.0,-14.163883,-16.587612,0.0,0.0,0.0,0.24,3.0,1016.900024,987.389832,100.0,99.0,100.0,0.0,9.178235,281.309906,23.039999,2.0
45148,2026-02-25 04:00:00,-12.660999,83.0,-14.946757,-17.409317,0.0,0.0,0.0,0.24,3.0,1017.099976,987.478271,100.0,100.0,81.0,8.0,7.993298,277.765076,17.280001,2.0


In [130]:
# All variables forecast
print('Predicting all variables forecast')
all_variable_df = add_holiday_flags(df_with_staffing, ts_col='ds', include_names=True).merge(weather_df, on='ds')

forecast_all_vars_with_future = pipeline.predict_df(
    all_variable_df,
    prediction_length=24,
    #future_df should be future_df_with_staffing merged with future_weather_df on 'ds' and 'id'
    future_df = future_df_with_staffing.merge(future_weather_df, on=['ds', 'id']),
    # quantile_levels=[0.1, 0.5, 0.9],
    quantile_levels=[0.2,0.5,0.8],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)   
forecast_all_vars_with_future.head()

Predicting all variables forecast


,id,ds,target_name,predictions,0.2,0.5,0.8
0,jgh,2026-02-18 22:00:00,INFLOW_STRETCHER,4.961423,3.312035,4.961423,6.944698
1,jgh,2026-02-18 23:00:00,INFLOW_STRETCHER,4.229599,2.619837,4.229599,6.192960
2,jgh,2026-02-19 00:00:00,INFLOW_STRETCHER,3.600605,2.175506,3.600605,5.392986
3,jgh,2026-02-19 01:00:00,INFLOW_STRETCHER,3.161630,1.771921,3.161630,4.824776
4,jgh,2026-02-19 02:00:00,INFLOW_STRETCHER,2.462353,1.343251,2.462353,3.962524


In [131]:
#join the predictions columns of basic_forecast, forecast_with_holidays, forecast_with_staffing, forecast_with_weather, forecast_all_vars_without_future, forecast_all_vars_with_future on the 'ds' column
basic_forecast = basic_forecast[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'basic_forecast'})
forecast_with_holidays = forecast_with_holidays[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_holidays'})
forecast_with_staffing = forecast_with_staffing[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_staffing'})
forecast_with_weather = forecast_with_weather[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_with_weather'})
# forecast_all_vars_without_future = forecast_all_vars_without_future[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_all_vars_without_future'})
forecast_all_vars_with_future_to_merge = forecast_all_vars_with_future[['ds', 'target_name', 'predictions']].rename(columns={'predictions':'forecast_all_vars_with_future'})

pred_df = basic_forecast.merge(forecast_with_holidays, on=['ds', 'target_name']).merge(forecast_with_staffing, on=['ds', 'target_name']).merge(forecast_with_weather, on=['ds', 'target_name']).merge(forecast_all_vars_with_future_to_merge, on=['ds', 'target_name'])
pred_df.head()

,ds,target_name,basic_forecast,forecast_with_holidays,forecast_with_staffing,forecast_with_weather,forecast_all_vars_with_future
0,2026-02-18 22:00:00,INFLOW_STRETCHER,5.088941,5.051457,5.014192,5.068197,4.961423
1,2026-02-18 23:00:00,INFLOW_STRETCHER,4.261419,4.246370,4.284819,4.245958,4.229599
2,2026-02-19 00:00:00,INFLOW_STRETCHER,3.503133,3.524253,3.757114,3.512157,3.600605
3,2026-02-19 01:00:00,INFLOW_STRETCHER,2.976579,2.988098,3.379319,3.008778,3.161630
4,2026-02-19 02:00:00,INFLOW_STRETCHER,2.313057,2.340688,2.601812,2.382319,2.462353


In [132]:
# Create a new dataframe with the average % difference of each forecast compared to the basic forecast, for each target_name averaged over all ds
# The output dataframe should only have 1 row per target_name, and columns for the average % difference of each forecast compared to the basic forecast
comparison_df = pred_df.copy()
comparison_df['%diff_holidays'] = (comparison_df['forecast_with_holidays'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_staffing'] = (comparison_df['forecast_with_staffing'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_weather'] = (comparison_df['forecast_with_weather'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df['%diff_all_vars_with_future'] = (comparison_df['forecast_all_vars_with_future'] - comparison_df['basic_forecast']) / comparison_df['basic_forecast'] * 100
comparison_df = comparison_df.groupby('target_name')[['%diff_holidays', '%diff_staffing', '%diff_weather', '%diff_all_vars_with_future']].mean().reset_index()
comparison_df.head()
# comparison_df.to_csv('forecast_variable_effects.csv', index=False)

,target_name,%diff_holidays,%diff_staffing,%diff_weather,%diff_all_vars_with_future
0,AMBVERT1,-0.287650,9.373008,5.968262,14.320342
1,AMBVERTTBS,0.064203,12.381347,5.612082,16.183466
2,CUM_ADMREQ,0.058373,0.895689,-3.686250,-0.278194
3,CUM_BA1,0.094399,6.306149,0.834570,9.125175
4,FLS,-49.396809,-54.065777,33.851810,-39.964554


In [133]:
anomaly_detection_ranges_df = pd.read_csv('https://www.dropbox.com/scl/fi/fjz0am427gw35sz7l994m/anomaly_detection_ranges.csv?rlkey=lib9w0jz2zei5n566jv76o7ol&raw=1')
anomaly_detection_ranges_df.ds = pd.to_datetime(anomaly_detection_ranges_df.ds, errors="coerce")
anomaly_detection_ranges_df.tail()

,ds,INFLOW_STRETCHER_yhat,INFLOW_STRETCHER_yhat_lower,INFLOW_STRETCHER_yhat_upper,Infl_Stretcher_cum_yhat,Infl_Stretcher_cum_yhat_lower,Infl_Stretcher_cum_yhat_upper,INFLOW_AMBULATORY_yhat,INFLOW_AMBULATORY_yhat_lower,INFLOW_AMBULATORY_yhat_upper,...,total_tbs_yhat_upper,vert_tbs_yhat,vert_tbs_yhat_lower,vert_tbs_yhat_upper,pod_tbs_yhat,pod_tbs_yhat_lower,pod_tbs_yhat_upper,overflow_yhat,overflow_yhat_lower,overflow_yhat_upper
331,2026-02-19 21:00:00,6.696714,1.802424,11.619308,139.707146,101.901821,177.656871,4.352476,-0.203400,8.953634,...,41.994893,18.867586,3.846503,34.279388,5.744107,0.634606,10.332182,11.145015,2.151968,19.514478
332,2026-02-19 22:00:00,6.121090,0.945304,10.982407,150.280560,115.100569,186.773367,3.820164,-1.089628,8.876808,...,38.589973,17.573136,2.665570,31.795491,5.644985,0.685674,10.533595,10.749878,2.870187,19.433805
333,2026-02-19 23:00:00,5.286725,0.190287,9.803927,139.332934,102.146741,176.226473,3.074107,-1.897070,7.819009,...,38.648552,16.934022,3.057174,32.153232,5.390424,0.502800,10.311826,10.425133,2.793916,18.719313
334,2026-02-20 00:00:00,4.413434,-0.472789,9.255871,103.442195,69.143827,141.911745,2.346290,-2.773706,7.528913,...,38.043533,17.108873,3.052264,32.388837,4.910608,-0.175624,10.142952,10.125728,2.354540,18.099791
335,2026-02-20 01:00:00,3.722666,-1.637739,8.385779,56.196857,17.703664,93.141863,1.851982,-3.068016,6.753954,...,37.470679,17.550123,3.339559,33.297021,4.254566,-0.708479,9.695353,9.853741,1.596554,18.026973


In [134]:
recent_df = df.tail(24).copy()
targets = recent_df.columns.tolist()
#remove ds and id from targets
targets = [t for t in targets if t not in ['ds', 'id']]

# Merge recent_df with anomaly_detection_ranges_df on 'ds' to align the data
recent_df = recent_df.merge(anomaly_detection_ranges_df, on='ds', how='left')

for target in targets:
    recent_df[target+'_hist_anomaly'] = ((recent_df[target] < recent_df[target+'_yhat_lower']) | (recent_df[target] > recent_df[target+'_yhat_upper'])).map({True: 'yes', False: 'no'})
    recent_df[target+'_hist_colour'] = recent_df.apply(lambda row: '#D13438' if row[target+'_hist_anomaly'] == 'yes' else ('#FFB900' if row[target] > row[target+'_yhat'] else ('#107C10' if row[target] < row[target+'_yhat'] else '#000000')), axis=1)

#remove all the columns containing yhat, yhat_lower, yhat_upper from recent_df
recent_df = recent_df[[col for col in recent_df.columns if not any(sub in col for sub in ['yhat', 'yhat_lower', 'yhat_upper'])]]
recent_df.columns.tolist()

/tmp/ipykernel_555394/3475870156.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  recent_df[target+'_hist_anomaly'] = ((recent_df[target] < recent_df[target+'_yhat_lower']) | (recent_df[target] > recent_df[target+'_yhat_upper'])).map({True: 'yes', False: 'no'})
/tmp/ipykernel_555394/3475870156.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  recent_df[target+'_hist_colour'] = recent_df.apply(lambda row: '#D13438' if row[target+'_hist_anomaly'] == 'yes' else ('#FFB900' if row[target] > row[target+'_yhat'] else ('#107C10' 

['ds',
 'INFLOW_STRETCHER',
 'Infl_Stretcher_cum',
 'INFLOW_AMBULATORY',
 'Infl_Ambulatory_cum',
 'Inflow_Total',
 'Inflow_Cum_Total',
 'INFLOW_AMBULANCES',
 'Infl_Ambulances_cum',
 'FLS',
 'CUM_ADMREQ',
 'CUM_BA1',
 'WAITINGADM',
 'TTStr',
 'TRG_HALLWAY1',
 'TRG_HALLWAY_TBS',
 'reoriented_cum',
 'reoriented_cum_MD',
 'QTRACK1',
 'RESUS',
 'Pod_T',
 'POD_GREEN',
 'POD_GREEN_TBS',
 'POD_YELLOW',
 'POD_YELLOW_TBS',
 'POD_ORANGE',
 'POD_ORANGE_TBS',
 'POD_CONS_MORE2H',
 'POD_IMCONS_MORE4H',
 'POD_XRAY_MORE2H',
 'POD_CT_MORE2H',
 'POST_POD1',
 'VERTSTRET',
 'RAZ_TBS',
 'RAZ_LAZYBOY',
 'RAZ_WAITINGREZ',
 'AMBVERT1',
 'AMBVERTTBS',
 'QTrack_TBS',
 'Garage_TBS',
 'RAZ_CONS_MORE2H',
 'RAZ_IMCONS_MORE4H',
 'RAZ_XRAY_MORE2H',
 'RAZ_CT_MORE2H1',
 'PSYCH1',
 'PSYCH_WAITINGADM',
 'total_tbs',
 'vert_tbs',
 'pod_tbs',
 'overflow',
 'id',
 'INFLOW_STRETCHER_hist_anomaly',
 'INFLOW_STRETCHER_hist_colour',
 'Infl_Stretcher_cum_hist_anomaly',
 'Infl_Stretcher_cum_hist_colour',
 'INFLOW_AMBULATORY_hist_a

In [137]:
targets = pred_df['target_name'].unique().tolist()
output_df = pd.DataFrame()
for target in targets:
    target_df = forecast_all_vars_with_future[forecast_all_vars_with_future['target_name'] == target][['ds', 'predictions', '0.2', '0.8']].rename(columns={'predictions': target+'_forecast', '0.2': target+'_forecast_lower', '0.8': target+'_forecast_upper'})
    # target_df = pred_df[pred_df['target_name'] == target][['ds', 'forecast_all_vars_with_future']].rename(columns={'forecast_all_vars_with_future': target+'_forecast'})
    target_df = target_df.merge(anomaly_detection_ranges_df[['ds', target+'_yhat', target+'_yhat_lower', target+'_yhat_upper']], on=['ds'], how='left')
    target_df[target+'_anomaly'] = ((target_df[target+'_forecast'] < target_df[target+'_yhat_lower']) | (target_df[target+'_forecast'] > target_df[target+'_yhat_upper'])).map({True: 'yes', False: 'no'})
    #assign a colour based on how the value compares to the yhat and the yhat_lower and yhat_upper. If it's an anomaly, colour is #D13438. If it's between yhat and yhat_upper, colour is #FFB900. If it's between yhat_lower and yhat, colour is #107C10. 
    target_df[target+'_colour'] = target_df.apply(lambda row: '#D13438' if row[target+'_anomaly'] == 'yes' else ('#FFB900' if row[target+'_forecast'] > row[target+'_yhat'] else '#107C10'), axis=1)
    #remove all the columns containing yhat, yhat_lower, yhat_upper from recent_df
    target_df = target_df[[col for col in target_df.columns if not any(sub in col for sub in ['yhat', 'yhat_lower', 'yhat_upper'])]]
    if output_df.empty:
        output_df = target_df
    else:
        output_df = output_df.merge(target_df, on='ds', how='outer')

output_df = output_df.merge(recent_df, on='ds', how='outer')

#Merge output_df with anomaly_detection_ranges_df on 'ds' to align the data, but only keep the rows where ds is in output_df
output_df = output_df.merge(anomaly_detection_ranges_df, on='ds', how='inner')


output_df.to_csv('ED_Hourly_Forecasts_Anomalies_v1.0.csv', index=False)

output_df.head()
    

,ds,INFLOW_STRETCHER_forecast,INFLOW_STRETCHER_forecast_lower,INFLOW_STRETCHER_forecast_upper,INFLOW_STRETCHER_anomaly,INFLOW_STRETCHER_colour,Infl_Stretcher_cum_forecast,Infl_Stretcher_cum_forecast_lower,Infl_Stretcher_cum_forecast_upper,Infl_Stretcher_cum_anomaly,...,total_tbs_yhat_upper,vert_tbs_yhat,vert_tbs_yhat_lower,vert_tbs_yhat_upper,pod_tbs_yhat,pod_tbs_yhat_lower,pod_tbs_yhat_upper,overflow_yhat,overflow_yhat_lower,overflow_yhat_upper
0,2026-02-17 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.247859,19.675736,4.803896,34.925069,5.617032,0.773508,10.750939,11.668779,3.699906,19.438464
1,2026-02-17 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.091638,18.990372,4.610096,34.469920,5.377599,0.413880,10.489035,11.348112,3.147325,18.992344
2,2026-02-18 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.869599,19.109206,4.174672,33.592061,4.913956,-0.189106,9.967105,11.053430,2.890283,19.053658
3,2026-02-18 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.747447,19.484461,4.903473,34.671270,4.275037,-0.826553,9.206807,10.787105,2.803000,19.465102
4,2026-02-18 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.492743,19.094947,3.818731,34.736371,3.589682,-1.476417,8.715166,10.580110,1.962886,18.648669


In [138]:
output_df.tail()

,ds,INFLOW_STRETCHER_forecast,INFLOW_STRETCHER_forecast_lower,INFLOW_STRETCHER_forecast_upper,INFLOW_STRETCHER_anomaly,INFLOW_STRETCHER_colour,Infl_Stretcher_cum_forecast,Infl_Stretcher_cum_forecast_lower,Infl_Stretcher_cum_forecast_upper,Infl_Stretcher_cum_anomaly,...,total_tbs_yhat_upper,vert_tbs_yhat,vert_tbs_yhat_lower,vert_tbs_yhat_upper,pod_tbs_yhat,pod_tbs_yhat_lower,pod_tbs_yhat_upper,overflow_yhat,overflow_yhat_lower,overflow_yhat_upper
43,2026-02-19 17:00:00,7.347727,5.005892,10.090567,no,#107C10,116.451340,104.376999,129.541077,no,...,48.514949,26.008433,10.446404,40.688219,7.063841,1.750426,12.261523,14.058630,6.254076,22.249723
44,2026-02-19 18:00:00,6.306870,4.049844,8.958897,no,#107C10,122.026421,109.995018,134.584564,no,...,46.807410,23.954285,8.323962,38.791288,6.541525,1.870545,11.438830,13.263619,5.433771,21.944633
45,2026-02-19 19:00:00,6.243613,3.982340,8.775322,no,#107C10,125.894661,114.850441,137.915009,no,...,45.038282,22.056149,8.300039,37.318020,6.097684,0.670702,11.015705,12.411198,4.721783,20.787892
46,2026-02-19 20:00:00,6.104573,3.820734,8.673702,no,#107C10,131.616364,120.536102,143.773773,no,...,43.469375,20.406472,6.412923,36.018176,5.845057,0.795069,10.783863,11.684792,3.543379,19.646832
47,2026-02-19 21:00:00,5.563436,3.474145,8.081989,no,#107C10,135.568207,124.288635,148.551178,no,...,41.994893,18.867586,3.846503,34.279388,5.744107,0.634606,10.332182,11.145015,2.151968,19.514478


In [139]:
dropbox_app_key = os.environ.get("DROPBOX_APP_KEY")
dropbox_app_secret = os.environ.get("DROPBOX_APP_SECRET")
dropbox_refresh_token = os.environ.get("DROPBOX_REFRESH_TOKEN")

# exchange the authorization code for an access token:
token_url = "https://api.dropboxapi.com/oauth2/token"
params = {
    "grant_type": "refresh_token",
    "refresh_token": dropbox_refresh_token,
    "client_id": dropbox_app_key,
    "client_secret": dropbox_app_secret
}
r = requests.post(token_url, data=params)

dropbox_access_token = r.json()['access_token']

dbx = dropbox.Dropbox(dropbox_access_token)

upload(dbx, 'chronos_forecast.csv', '', '',
            'chronos_forecast.csv', overwrite=True)
upload(dbx, 'ED_Hourly_Forecasts_Anomalies_v1.0.csv', '', '',
            'ED_Hourly_Forecasts_Anomalies_v1.0.csv', overwrite=True)
upload(dbx, 'forecast_variable_effects.csv', '', '',
            'forecast_variable_effects.csv', overwrite=True)

uploaded as b'chronos_forecast.csv'
uploaded as b'ED_Hourly_Forecasts_Anomalies_v1.0.csv'
uploaded as b'forecast_variable_effects.csv'


FileMetadata(client_modified=datetime.datetime(2026, 2, 16, 20, 46, 58), content_hash='e67091084ccd8e1bdd5edfb96548e0e89d3c74f2d3983d49269226560633e641', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:oNSmVCFixyQAAAAAAABxQQ', is_downloadable=True, media_info=NOT_SET, name='forecast_variable_effects.csv', parent_shared_folder_id=NOT_SET, path_display='/forecast_variable_effects.csv', path_lower='/forecast_variable_effects.csv', preview_url=NOT_SET, property_groups=NOT_SET, rev='64b23e4fb21737a19c0a3', server_modified=datetime.datetime(2026, 2, 19, 2, 16, 37), sharing_info=NOT_SET, size=2710, symlink_info=NOT_SET)